In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/transaction-data/transaction_data.csv
/kaggle/input/productcsv/product.csv


In [2]:
!pip install implicit

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    

In [4]:
data = pd.read_csv('../input/transaction-data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


In [5]:
item_features = pd.read_csv('../input/productcsv/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [6]:
item_features.department.unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

In [7]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [8]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [9]:
data_train.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [10]:
# Заведем фиктивный item_id

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999_999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix)

user_item_matrix.head(3)

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [12]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [13]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    flags = np.isin(bought_list, recommended_list)
    #print(flags)
    precision = flags.sum() / len(recommended_list)
    return precision

In [14]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [15]:
%%time

# model = AlternatingLeastSquares(factors=128, 
#                                 regularization=0.05,
#                                 iterations=15, 
#                                 calculate_training_loss=True, 
#                                 num_threads=4) # K - кол-во билжайших соседей

model = AlternatingLeastSquares(factors=64, 
                                regularization=0.05,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=10)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 1min 32s, sys: 1min 14s, total: 2min 46s
Wall time: 42.5 s


In [16]:
result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()

0.17498744349572826

In [17]:
#нужно было конечно сразу от bm_25 взвешивания идти. 

In [18]:
factors = np.arange(16, 224, 16)

In [19]:
factors

array([ 16,  32,  48,  64,  80,  96, 112, 128, 144, 160, 176, 192, 208])

In [26]:
regularizations = np.arange(0.0001, 0.012, 0.0005)

In [27]:
regularizations

array([0.0001, 0.0006, 0.0011, 0.0016, 0.0021, 0.0026, 0.0031, 0.0036,
       0.0041, 0.0046, 0.0051, 0.0056, 0.0061, 0.0066, 0.0071, 0.0076,
       0.0081, 0.0086, 0.0091, 0.0096, 0.0101, 0.0106, 0.0111, 0.0116])

In [28]:
for num_factors in factors: 
    for num_regs in regularizations: 
        model = AlternatingLeastSquares(factors=num_factors, 
                                    regularization=num_regs,
                                    iterations=15, 
                                    calculate_training_loss=True, 
                                    num_threads=10
                                    ) 

        model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=True)

        recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                            user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                            N=5, # кол-во рекомендаций 
                            filter_already_liked_items=False, 
                            filter_items=None, 
                            recalculate_user=True)

        result[f'num_factors={num_factors}_reg={num_regs}'] = \
                            result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

...

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

In [29]:
%%time
    
result.apply(lambda row: precision_at_k(row[f'num_factors={num_factors}_reg={num_regs}'], \
                                                            row['actual']), axis=1).mean()

CPU times: user 271 ms, sys: 0 ns, total: 271 ms
Wall time: 269 ms


0.1906579608237039

In [31]:
result.head(2)

,user_id,actual,als_bm25,num_factors=16_reg=0.0001,num_factors=16_reg=0.0006000000000000001,num_factors=16_reg=0.0011,num_factors=16_reg=0.0016,num_factors=16_reg=0.0021,num_factors=16_reg=0.0026,num_factors=16_reg=0.0031,...,num_factors=208_reg=0.0071,num_factors=208_reg=0.0076,num_factors=208_reg=0.0081,num_factors=208_reg=0.0086,num_factors=208_reg=0.0091,num_factors=208_reg=0.0096,num_factors=208_reg=0.0101,num_factors=208_reg=0.0106,num_factors=208_reg=0.011099999999999999,num_factors=208_reg=0.0116
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[999999, 995242, 1100972, 1082185, 5569374]","[999999, 1082185, 981760, 960732, 1104349]","[999999, 1082185, 1004390, 981760, 1045586]","[999999, 1082185, 1104349, 1004390, 999391]","[999999, 1082185, 1004390, 960732, 1104349]","[999999, 1082185, 1082212, 981760, 885290]","[999999, 1082185, 981760, 885290, 878996]","[999999, 1082185, 1004390, 981760, 960732]",...,"[1082185, 995242, 999999, 965766, 934369]","[995242, 999999, 1082185, 965766, 1033142]","[1082185, 995242, 999999, 1033142, 965766]","[1082185, 995242, 999999, 1033142, 904360]","[1082185, 995242, 999999, 965766, 934369]","[1082185, 999999, 995242, 965766, 1100972]","[1082185, 999999, 995242, 965766, 934369]","[995242, 1082185, 999999, 965766, 934369]","[999999, 1082185, 995242, 965766, 1033142]","[965766, 1082185, 999999, 995242, 934369]"
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[999999, 951590, 840601, 844165, 1053690]","[883404, 844165, 999999, 951590, 826249]","[8090537, 844165, 951590, 999999, 8090521]","[8090521, 999999, 8090537, 883404, 5568378]","[844165, 999999, 883404, 8090537, 1029743]","[8090521, 999999, 8090537, 962229, 883404]","[8090537, 999999, 951590, 8090521, 5569230]","[999999, 951590, 844165, 883404, 8090521]",...,"[1133018, 883404, 908531, 1022003, 1106523]","[1133018, 1022003, 883404, 1092026, 999999]","[1133018, 1092026, 999999, 1106523, 914190]","[1133018, 999999, 1092026, 1022003, 1044078]","[1133018, 914190, 999999, 1092026, 1053690]","[1133018, 1022003, 914190, 1092026, 999999]","[1133018, 883404, 1092026, 999999, 914190]","[1133018, 999999, 1092026, 914190, 883404]","[1133018, 1092026, 999999, 1106523, 1053690]","[1133018, 1106523, 999999, 1092026, 883404]"


In [30]:
df_res_2 = pd.DataFrame(columns=['parameters', 'presision_at_k'])
i = 0
for column in result.columns[2:]:
    a = round(result.apply(lambda row: precision_at_k(row[column], row['actual']), axis=1).mean(), 3)
    #print(column, a)
    #df = pd.DataFrame([column, a])
    df_res_2.loc[i] = [column, a]
    i += 1

In [32]:
df_res_2.sort_values(by='presision_at_k', ascending=False).head(10)

,parameters,presision_at_k
286,num_factors=192_reg=0.0106,0.200
270,num_factors=192_reg=0.0026,0.200
288,num_factors=192_reg=0.0116,0.199
233,num_factors=160_reg=0.0081,0.199
257,num_factors=176_reg=0.0081,0.198
266,num_factors=192_reg=0.0006000000000000001,0.198
207,num_factors=144_reg=0.0071,0.198
251,num_factors=176_reg=0.0051,0.198
248,num_factors=176_reg=0.0036,0.198
300,num_factors=208_reg=0.0056,0.197
